# Introducción al uso de interfaces de datos (API)
## ¿Qué es un API?

Un **API (Application Programming Interface)** es un interfaz que permite que dos aplicaciones o sistemas se comuniquen entre sí. A través de un API, un programa puede solicitar datos o servicios de otro sin necesidad de que el usuario interactúe directamente con ambas aplicaciones. Las APIs facilitan el desarrollo de aplicaciones de explotación de datos (IA), ya que permiten la integración y el acceso a servicios y datos de manera eficiente.

## Tipos de consulta en un API REST

Las APIs REST (Representational State Transfer) son un tipo de API que permite a los sistemas interactuar a través de la web utilizando los métodos HTTP estándar. A continuación, los tipos de consultas más comunes en una API REST:

1. **GET**: Se utiliza para **solicitar** datos de un recurso específico en el servidor. Es el método más común cuando queremos obtener información.
   - Ejemplo: Obtener datos meteorológicos actuales.
2. **POST**: Se utiliza para **enviar** datos al servidor, normalmente para crear un nuevo recurso.
   - Ejemplo: Enviar datos de usuario para registrarse en un servicio.
3. **PUT**: Se utiliza para **actualizar** un recurso existente en el servidor con datos proporcionados.
   - Ejemplo: Actualizar información de perfil de usuario.
4. **DELETE**: Se utiliza para **eliminar** un recurso existente en el servidor.
   - Ejemplo: Eliminar un usuario de un sistema.

In [ ]:
! pip install requests

---

## API de OpenMeteo

**OpenMeteo** es una API pública que proporciona datos meteorológicos sin necesidad de registro o autenticación. Esto facilita mucho el proceso de acceso, ya que no se requiere una cuenta o clave API.

1. **Acceder al sitio web**: [https://open-meteo.com/](https://open-meteo.com/)
2. **Documentación**: La API tiene una documentación accesible donde se describen los diferentes endpoints y parámetros que puedes utilizar para hacer consultas (por ejemplo, coordenadas geográficas, intervalo de tiempo, variables meteorológicas).
3. **Consulta de datos**: Puedes realizar peticiones directamente a la API utilizando los parámetros de consulta correctos.

---

## **Ejercicio: Consulta sencilla a OpenMeteo**

Las coordenadas geográficas de **Icod de Los Vinos** son:
- **Latitud**: 28.367
- **Longitud**: -16.721

Vamos a realizar una consulta a la API de OpenMeteo para obtener el pronóstico de temperatura utilizando estas coordenadas:

In [ ]:
import requests
import json

# Coordenadas de Icod de Los Vinos
latitud = 28.367
longitud = -16.721

# URL del API de OpenMeteo para el pronóstico del tiempo
url = f"https://api.open-meteo.com/v1/forecast?latitude={latitud}&longitude={longitud}&hourly=temperature_2m"

# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta en JSON
    data = response.json()
    data
else:
    data = None
    print(f"Error en la solicitud: {response.status_code}")


### Explicación del código:
* Parámetros de consulta:
  * Latitud y longitud de Icod de Los Vinos se insertan en la URL de la API.
  * El parámetro hourly=temperature_2m especifica que queremos obtener el pronóstico de temperatura a 2 metros del suelo, en intervalos horarios.
* Solicitud HTTP:
  * Se realiza una petición GET a la API de OpenMeteo utilizando la librería requests. La respuesta del servidor se almacena en la variable response si la respuesta es correcta (200).

A continuación mostramos los datos obtenidos en formato JSON.

In [ ]:
data

Como podemos observar la consulta nos devuelve una predicción de los valores de temperatura en las coordenadas espcficadas para los siguientes 6 días a una frecuencia horaria.

### Guardado de datos en formato JSON
Antes de seguir, para evitar perder los datos de la consulta al API, los guardaremos en local usando para ello el formato json.

In [ ]:
 # Guardar la respuesta en un archivo JSON
with open('data/m2_e1_datos_meteo.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)
    print("Datos meteorológicos guardados en 'icod_meteo.json'.")

### Lectura de archivo en formato JSON
Ahora volvemos a leer los datos del archivo guardado en disco y lo transformamos en un objeto de pandas:

In [ ]:
import pandas as pd

# Extraer las horas y las temperaturas
horas = data['hourly']['time']
temperaturas = data['hourly']['temperature_2m']

# Crear un DataFrame con las horas y las temperaturas
df = pd.DataFrame({
    'Fecha': pd.to_datetime(horas),  # Convertir las horas a formato datetime
    'Temperatura (°C)': temperaturas
})

# Mostrar las primeras filas del DataFrame
print(df.head())

Una vez que tienes los datos en el DataFrame, procedemos a hacer un análisi descrptivo de los datos

In [ ]:
# Ver estadísticas descriptivas del DataFrame
print(df.describe())

In [ ]:
print(df.info())

In [ ]:
# Guardar el DataFrame en un archivo CSV (opcional)
df.to_csv('data/m2_e1_datos_meteo.csv', index=False)

### Visulización básica de los datos

#### Gráfico de línea
Visualizamos la serie temporal de temperaturas en un gráfico de línea, añadiendo una línea discontinua que represente el promedio de la serie 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar los datos desde el archivo CSV (o usa el DataFrame directamente si ya está creado)
df = pd.read_csv('data/m2_e1_datos_meteo.csv')

# Convertir la columna 'Fecha' al tipo datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
plt.plot(df['Fecha'], df['Temperatura (°C)'], linestyle='-', marker='o', markersize=3)

# Calcular el promedio, máximo y mínimo de la columna 'Temperatura (°C)'
promedio_temperatura = df['Temperatura (°C)'].mean()
# Añadir línea discontinua para el promedio
plt.axhline(y=promedio_temperatura, color='red', linestyle='--', linewidth=2, label=f'Promedio: {promedio_temperatura:.2f} °C')

# Añadir etiquetas y título
plt.title('Serie Temporal de Temperatura en Icod de Los Vinos', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Temperatura (°C)', fontsize=12)

# Rotar las etiquetas del eje X para mejor legibilidad
plt.xticks(rotation=45)

# Mostrar una cuadrícula
plt.grid(True)

# Ajustar los márgenes
plt.tight_layout()

# Mostrar el gráfico
plt.show()


#### Histograma
Generamos un histograma de la serie de temperaturas, que nos permitirá visualizar la distribución de las temperaturas en la serie temporal:

In [ ]:
# Crear el histograma de la columna 'Temperatura (°C)'
plt.figure(figsize=(8, 6))
plt.hist(df['Temperatura (°C)'], bins=10, color='skyblue', edgecolor='black')

# Añadir etiquetas y título
plt.title('Histograma de Temperaturas en Icod de Los Vinos', fontsize=16)
plt.xlabel('Temperatura (°C)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

# Mostrar el histograma
plt.grid(True)
plt.tight_layout()
plt.show()


**Comentarios:**
* Distribución de las temperaturas: El histograma te mostrará cómo se distribuyen las temperaturas a lo largo del tiempo. Si el histograma tiene una forma de campana, podría indicar una distribución normal en las temperaturas.
* Frecuencia de temperaturas: Te permitirá ver si hay rangos de temperatura que ocurren con mayor frecuencia. Por ejemplo, puede que encuentres que la mayor parte de las temperaturas están entre 15 y 20°C, lo que podría ser la temperatura más común en el periodo analizado.
* Posibles extremos: Si hay barras en los extremos del gráfico con una frecuencia muy baja, podrían corresponder a valores extremos o inusuales de temperatura.